This notebook uses an R kernel.

# RNA-seq Raw Count Matrix Preprocessing

Author: Zhongyi (James) Guo <br>
Date: 10/28/2024

## Import Packages

In [1]:
getwd()

[1] "/Users/zhongyiguo/Desktop/SCA-DL-DGE/script/deseq2"

In [2]:
.libPaths()

[1] "/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library"

In [3]:
library(tidyverse)
library(GEOquery)
library(AnnoProbe)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, 

## Data Wrangling

### Meta Data

In [4]:
eSet <- getGEO("GSE248760", destdir = '.', getGPL = F)

Found 1 file(s)

GSE248760_series_matrix.txt.gz



In [5]:
title <- eSet$GSE248760_series_matrix.txt.gz@phenoData$title
title

[1] "Sample_1, IS" "Sample_2, IS" "Sample_3, IS" "Sample_4, IS" "Sample_5, C" 
[6] "Sample_6, C"  "Sample_7, C"  "Sample_8, C"

In [6]:
meta_data <- title |>
    as.data.frame() |>
    separate(title, into = c("sample", "status"), sep = ", ")
meta_data

sample,status
<chr>,<chr>
Sample_1,IS
Sample_2,IS
Sample_3,IS
Sample_4,IS
Sample_5,C
Sample_6,C
Sample_7,C
Sample_8,C


In [7]:
write_csv(meta_data, '../../result/deseq2/meta_data.csv')

## Count Matrix

The RNA-seq raw count matrix was downloaded from GEO using `http`: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE248760

In [8]:
count <- read.table('../../data/GSE248760_raw_counts.txt', header = TRUE)
head(count)

,NAME,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CLASS:DOSE,IS,IS,IS,IS,C,C,C,C
2,ENSG00000223972,13,18,3,1,5,11,17,10
3,ENSG00000227232,1087,1002,182,531,200,114,319,172
4,ENSG00000278267,23,33,1,12,6,1,4,1
5,ENSG00000243485,0,3,0,1,2,2,0,2
6,ENSG00000284332,0,0,0,0,0,0,0,0


The first row indicates the sample status for ischemic stroke (IS) or not (C, control). This piece of information has already been included in meta data, so we will remove it.

In [9]:
count <- count |> slice(-1)
count

NAME,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ENSG00000223972,13,18,3,1,5,11,17,10
ENSG00000227232,1087,1002,182,531,200,114,319,172
ENSG00000278267,23,33,1,12,6,1,4,1
ENSG00000243485,0,3,0,1,2,2,0,2
ENSG00000284332,0,0,0,0,0,0,0,0
ENSG00000237613,0,0,0,0,0,0,2,0
ENSG00000268020,0,0,0,0,0,0,0,0
ENSG00000240361,0,0,0,0,0,0,0,0
ENSG00000186092,0,0,0,0,0,0,0,0


In [10]:
write_tsv(count, '../../result/deseq2/count_clean.tsv')

## Conclusion

In this notebook, we extracted and cleaned the metadata and RNA-seq data for downstream differential gene expression analysis.

In [11]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: aarch64-apple-darwin20 (64-bit)
Running under: macOS 15.1

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.11.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/Los_Angeles
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] AnnoProbe_0.1.7     GEOquery_2.70.0     Biobase_2.62.0     
 [4] BiocGenerics_0.48.1 lubridate_1.9.3     forcats_1.0.0      
 [7] stringr_1.5.1       dplyr_1.1.4         purrr_1.0.2        
[10] readr_2.1.5         tidyr_1.3.1         tibble_3.2.1       
[13] ggplot2_3.5.0       tidyverse_2.0.0     jsonlite_1.8.8     
[16] formatR_1.14       

loaded via a namespace (and not attached):
 [1] gtable_0.3.4     